**Course Name：** DAT341 / DIT867 Applied Machine Learning

**Examiner：** Richard Johansson (richajo@chalmers.se)

**Assignment No.:** Assignment 1 - Introductory tour and Decision Trees

**Release Date:** Fri, 19 Jan 2024 0:00am

**Due Date:** Tue, 30 Jan 2024 11:59pm

**Group Name:** PA 2 3

**Group Members:**
- Natalia Alvarado (gusalvarsi@student.gu.se)
- Erdem Halil (gushaliler@student.gu.se)
- Xujie Li (guslixuf@student.gu.se)


In [ ]:
from google.colab import drive
import sys

drive.mount('/content/drive')

# Adjust the path to point to the folder in my Google Drive
folder_path = '/content/drive/My Drive/DIT867_AML/Assignment_2'
sys.path.append(folder_path)


Mounted at /content/drive


# **Task 1: Working with a dataset with categorical features**
## **Step 1. Reading the data**

## **Step 2: Encoding the features as numbers.**

## **Step 3. Combining the steps.**

# **Task 2: Decision trees and random forests**
## **Underfitting and overfitting in decision tree classifiers.**

## **Underfitting and overfitting in random forest classifiers.**

# **Task 3: Feature importances in random forest classifiers**